# Minalyze: Geochemistry

## Unsupervised segmentation w/ automated ML 

In [1]:
from pycaret.clustering import *
from sklearn.neighbors import kneighbors_graph

In [2]:
def get_variables(df):
    """List all variables"""
    value = list( df.columns )
    return value

def get_ignorefeatures(df):
    """List all ignored features"""
    value = [
        'id', 
        'result_master_id',
        'DDH_name',
        'from_m',
        'to_m',
        'Sample_Length_m',
        'Scan_Length_m',
        'Scan_Recovery_pct',
        'Comp(c/s)', 
        'Rayl(c/s)', 
        'LT(secs)',
        'minaloggerlink']
    additional = (
        [s for i, s in enumerate(get_variables(df)) if "mdl" in s] +
        [s for i, s in enumerate(get_variables(df)) if "2SE" in s]
        )
    value = value + additional
    return value

In [3]:
filepath   = '../data/OOLDEA2_1m_intervals.csv'
data = pd.read_csv( filepath )

In [4]:
exp = setup(data, normalize = True, 
    ignore_features = get_ignorefeatures(data),
    session_id = 123, 
    silent = True, 
    pca = False)     

,Description,Value
0,session_id,123
1,Original Data,"(156, 99)"
2,Missing Values,False
3,Numeric Features,27
4,Categorical Features,2
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(156, 29)"
9,CPU Jobs,-1


In [5]:
modeltype = "hclust"

 # Connectivity constraint for hierarchical cluster 
depth_constraint = data["from_m"].values.reshape(-1,1)
connect = kneighbors_graph(depth_constraint, n_neighbors=2, include_self=False)

constraints = { "connectivity": connect, 
    "affinity": 'euclidean', 
    "linkage": 'ward', 
    "compute_full_tree": True,
            }

model = create_model(modeltype, **constraints)

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.1775,16.0526,1.865,0,0,0


In [6]:
labeled = assign_model(model)
labeled.head()

,id,result_master_id,DDH_name,from_m,to_m,Sample_Length_m,Scan_Length_m,Scan_Recovery_pct,Comp(c/s),Rayl(c/s),...,Ce_2SE_ppm,Ce_mdl_ppm,Pb_pct,Pb_2SE_pct,Pb_mdl_pct,U_ppm,U_2SE_ppm,U_mdl_ppm,minaloggerlink,Cluster
0,69232,38,OOLDEA2,13,14,1,0.2401,24.01,2613.187,715.3073,...,54.90,40.99,0.0029,0.0017,0.0025,22.97,15.03,20.40,https://minalogger.com/viewer/?project=WESTERN...,Cluster 3
1,69233,38,OOLDEA2,14,15,1,0.4056,40.56,2498.827,653.7424,...,45.22,36.78,0.0030,0.0014,0.0021,20.18,12.16,16.75,https://minalogger.com/viewer/?project=WESTERN...,Cluster 3
2,69234,38,OOLDEA2,15,16,1,0.6329,63.29,2869.263,702.2439,...,36.45,30.90,0.0032,0.0012,0.0019,31.67,10.89,14.70,https://minalogger.com/viewer/?project=WESTERN...,Cluster 3
3,69235,38,OOLDEA2,16,17,1,0.6143,61.43,2594.439,641.5786,...,29.13,24.85,0.0036,0.0010,0.0015,24.23,9.12,12.22,https://minalogger.com/viewer/?project=WESTERN...,Cluster 3
4,69236,38,OOLDEA2,17,18,1,0.4486,44.86,2578.841,647.0553,...,28.67,26.67,0.0039,0.0011,0.0016,22.88,10.39,13.37,https://minalogger.com/viewer/?project=WESTERN...,Cluster 3


In [7]:
plot_model(model)